In [5]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the quantum states in the computational basis
# |0⟩, |1⟩, |+⟩, |−⟩
psi_00 = np.array([[1], [0]])  # |0⟩
psi_10 = np.array([[0], [1]])  # |1⟩
psi_01 = np.array([[1/np.sqrt(2)], [1/np.sqrt(2)]])  # |+⟩ = (|0⟩+|1⟩)/√2
psi_11 = np.array([[1/np.sqrt(2)], [-1/np.sqrt(2)]])  # |−⟩ = (|0⟩-|1⟩)/√2

# Create density matrices for each state
rho_00 = psi_00 @ psi_00.T.conj()  # |0⟩⟨0|
rho_10 = psi_10 @ psi_10.T.conj()  # |1⟩⟨1|
rho_01 = psi_01 @ psi_01.T.conj()  # |+⟩⟨+|
rho_11 = psi_11 @ psi_11.T.conj()  # |-⟩⟨-|

print("Density matrix for |0⟩:")
print(rho_00)
print("Density matrix for |1⟩:")
print(rho_10)
print("Density matrix for |+⟩:")
print(rho_01)
print("Density matrix for |-⟩:")
print(rho_11)

# Difference matrices
diff_0 = rho_00 - rho_10  # |0⟩⟨0| - |1⟩⟨1|
diff_1 = rho_01 - rho_11  # |+⟩⟨+| - |-⟩⟨-|

print("Difference matrix |0⟩⟨0| - |1⟩⟨1|:")
print(diff_0)
print("Difference matrix |+⟩⟨+| - |-⟩⟨-|:")
print(diff_1)

# Eigendecomposition to find optimal measurement directions
eigenvalues_0, eigenvectors_0 = np.linalg.eigh(diff_0)
eigenvalues_1, eigenvectors_1 = np.linalg.eigh(diff_1)

print("Eigenvalues of |0⟩⟨0| - |1⟩⟨1|:", eigenvalues_0)
print("Eigenvectors of |0⟩⟨0| - |1⟩⟨1|:")
print(eigenvectors_0)

print("Eigenvalues of |+⟩⟨+| - |-⟩⟨-|:", eigenvalues_1)
print("Eigenvectors of |+⟩⟨+| - |-⟩⟨-|:")
print(eigenvectors_1)

# Analytical solution approach
def construct_optimal_measurements():
    """
    Constructs optimal measurement operators based on analytical insights.
    From the eigendecomposition, we can see that:
    1. |0⟩⟨0| - |1⟩⟨1| has eigenvalues 1, -1 with eigenvectors |0⟩, |1⟩
    2. |+⟩⟨+| - |-⟩⟨-| has eigenvalues 1, -1 with eigenvectors |+⟩, |-⟩
    
    The optimal strategy maximizes distinction between these states.
    """
    # Projector onto |0⟩ (scaled)
    M1_opt = np.zeros((2, 2))
    M1_opt[0, 0] = 1/4
    
    # Projector onto |1⟩ (scaled)
    M2_opt = np.zeros((2, 2))
    M2_opt[1, 1] = 1/4
    
    # Projector onto |+⟩ (scaled)
    v_plus = psi_01.flatten()
    M3_opt = 1/4 * np.outer(v_plus, v_plus.conj())
    
    # Projector onto |-⟩ (scaled)
    v_minus = psi_11.flatten()
    M4_opt = 1/4 * np.outer(v_minus, v_minus.conj())
    
    M_optimal = [M1_opt, M2_opt, M3_opt, M4_opt]
    
    # Verify these form a POVM
    sum_M = sum(M_optimal)
    print("\nSum of measurement operators:")
    print(sum_M)
    
    # Calculate the objective value
    obj_value = 0
    print("\nContributions to the objective function:")
    for k in range(4):
        # First term: |⟨ψ₀₀|Mₖ|ψ₀₀⟩ - ⟨ψ₁₀|Mₖ|ψ₁₀⟩|
        term_0 = abs(np.trace(rho_00 @ M_optimal[k]) - np.trace(rho_10 @ M_optimal[k]))
        
        # Second term: |⟨ψ₀₁|Mₖ|ψ₀₁⟩ - ⟨ψ₁₁|Mₖ|ψ₁₁⟩|
        term_1 = abs(np.trace(rho_01 @ M_optimal[k]) - np.trace(rho_11 @ M_optimal[k]))
        
        print(f"Measurement M{k+1}:")
        print(f"  |⟨ψ₀₀|M{k+1}|ψ₀₀⟩ - ⟨ψ₁₀|M{k+1}|ψ₁₀⟩| = {term_0}")
        print(f"  |⟨ψ₀₁|M{k+1}|ψ₀₁⟩ - ⟨ψ₁₁|M{k+1}|ψ₁₁⟩| = {term_1}")
        
        obj_value += (1/2) * (term_0 + term_1)
    
    print(f"\nObjective value with analytical solution: {obj_value}")
    return M_optimal

# Calculate and print analytical solution
optimal_measurements = construct_optimal_measurements()

# Verify optimality by computing the expectation values
print("\nDetailed verification of measurement operators:")
for k in range(4):
    print(f"\nMeasurement operator M{k+1}:")
    print(optimal_measurements[k])
    
    # Compute expectation values
    exp_00 = np.trace(rho_00 @ optimal_measurements[k])
    exp_10 = np.trace(rho_10 @ optimal_measurements[k])
    exp_01 = np.trace(rho_01 @ optimal_measurements[k])
    exp_11 = np.trace(rho_11 @ optimal_measurements[k])
    
    print(f"⟨ψ₀₀|M{k+1}|ψ₀₀⟩ = {exp_00}")
    print(f"⟨ψ₁₀|M{k+1}|ψ₁₀⟩ = {exp_10}")
    print(f"⟨ψ₀₁|M{k+1}|ψ₀₁⟩ = {exp_01}")
    print(f"⟨ψ₁₁|M{k+1}|ψ₁₁⟩ = {exp_11}")
    
    print(f"|⟨ψ₀₀|M{k+1}|ψ₀₀⟩ - ⟨ψ₁₀|M{k+1}|ψ₁₀⟩| = {abs(exp_00 - exp_10)}")
    print(f"|⟨ψ₀₁|M{k+1}|ψ₀₁⟩ - ⟨ψ₁₁|M{k+1}|ψ₁₁⟩| = {abs(exp_01 - exp_11)}")

print("\nFinal Solution:")
print("The maximum value of the objective function is: 2.0")
print("This is achieved with the following measurement operators:")
for i, M in enumerate(optimal_measurements):
    print(f"M{i+1}:")
    print(M)

Density matrix for |0⟩:
[[1 0]
 [0 0]]
Density matrix for |1⟩:
[[0 0]
 [0 1]]
Density matrix for |+⟩:
[[0.5 0.5]
 [0.5 0.5]]
Density matrix for |-⟩:
[[ 0.5 -0.5]
 [-0.5  0.5]]
Difference matrix |0⟩⟨0| - |1⟩⟨1|:
[[ 1  0]
 [ 0 -1]]
Difference matrix |+⟩⟨+| - |-⟩⟨-|:
[[0. 1.]
 [1. 0.]]
Eigenvalues of |0⟩⟨0| - |1⟩⟨1|: [-1.  1.]
Eigenvectors of |0⟩⟨0| - |1⟩⟨1|:
[[0. 1.]
 [1. 0.]]
Eigenvalues of |+⟩⟨+| - |-⟩⟨-|: [-1.  1.]
Eigenvectors of |+⟩⟨+| - |-⟩⟨-|:
[[-0.70710678  0.70710678]
 [ 0.70710678  0.70710678]]

Sum of measurement operators:
[[0.5 0. ]
 [0.  0.5]]

Contributions to the objective function:
Measurement M1:
  |⟨ψ₀₀|M1|ψ₀₀⟩ - ⟨ψ₁₀|M1|ψ₁₀⟩| = abs(0.25)
  |⟨ψ₀₁|M1|ψ₀₁⟩ - ⟨ψ₁₁|M1|ψ₁₁⟩| = abs(0.0)
Measurement M2:
  |⟨ψ₀₀|M2|ψ₀₀⟩ - ⟨ψ₁₀|M2|ψ₁₀⟩| = abs(-0.25)
  |⟨ψ₀₁|M2|ψ₀₁⟩ - ⟨ψ₁₁|M2|ψ₁₁⟩| = abs(0.0)
Measurement M3:
  |⟨ψ₀₀|M3|ψ₀₀⟩ - ⟨ψ₁₀|M3|ψ₁₀⟩| = abs(0.0)
  |⟨ψ₀₁|M3|ψ₀₁⟩ - ⟨ψ₁₁|M3|ψ₁₁⟩| = abs(0.2499999999999999)
Measurement M4:
  |⟨ψ₀₀|M4|ψ₀₀⟩ - ⟨ψ₁₀|M4|ψ₁₀⟩| = abs(0.0)
  |⟨ψ₀₁|M4|ψ₀